#### Filtering low quality pixels

In [77]:
# Filtering pixels with high aerosol levels

import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Transformer
import zipfile
import os
import shutil

# Suppress SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # default='warn'

global filtered_gdf, df_filtered

def unzip_process_zip(file_zip_path, target_file, process_directory):
    global filtered_gdf, df_filtered
    
    # Ensure the processing directory is clear before extraction
    if os.path.exists(process_directory):
        shutil.rmtree(process_directory)
    os.makedirs(process_directory)
    
    # Extract all files in the zip to process directory
    with zipfile.ZipFile(file_zip_path, 'r') as z:
        z.extractall(process_directory)
        print("All files extracted for processing.")
        
    target_file_path = os.path.join(process_directory, target_file)

    # Check if the target file exists
    if not os.path.exists(target_file_path):
        print("Target file does not exist in the extracted files.")
        return

    # Process the extracted file
    with rasterio.open(target_file_path) as src:
        array = src.read()
        transform = src.transform

        # Create transformer object to convert from source CRS to WGS 84
        transformer = Transformer.from_crs(src.crs, 'EPSG:4326', always_xy=True)

        # Generate longitude and latitude coordinates
        cols, rows = np.meshgrid(np.arange(src.width), np.arange(src.height))
        xs, ys = rasterio.transform.xy(transform, rows, cols, offset='center')
        lon, lat = transformer.transform(np.array(xs).flatten(), np.array(ys).flatten())

        # Create DataFrame and convert to GeoDataFrame
        df = pd.DataFrame({'Longitude': lon, 'Latitude': lat})
        for i, band in enumerate(src.read(masked=True)):
            df[src.descriptions[i]] = band.flatten()

        # Convert 'SR_QA_AEROSOL' to integer for bitwise operation
        df['SR_QA_AEROSOL'] = df['SR_QA_AEROSOL'].astype(int)

        # Filter out pixels with valid aerosol retrieval and high aerosol level
        # Assuming 'SR_QA_AEROSOL' is the name of the QA aerosol band in the data
        valid_aerosol = (df['SR_QA_AEROSOL'] & 2) == 2  # Bit 1 must be set for valid retrieval
        high_aerosol = (df['SR_QA_AEROSOL'] & 192) == 192  # Bits 6-7 must be set to 11 for high aerosol
        filter_mask = valid_aerosol & high_aerosol
        df_filtered = df[-filter_mask]
        
        # Scale and offset specific bands
        df_filtered['ST_B10_Celsius'] = df_filtered['ST_B10'] * 0.00341802 + 149 - 273.15
        bands_to_scale = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
        for band in bands_to_scale:
            df_filtered[f"{band}_Scaled"] = df_filtered[band] * 2.75e-05 - 0.2

        additional_scales = {
            'ST_ATRAN': 0.0001, 'ST_CDIST': 0.01, 'ST_DRAD': 0.001, 
            'ST_EMIS': 0.0001, 'ST_EMSD': 0.0001, 'ST_QA': 0.01, 
            'ST_TRAD': 0.001, 'ST_URAD': 0.001
        }
        for band, scale in additional_scales.items():
            df_filtered[f"{band}_Scaled"] = df_filtered[band] * scale

        gdf = gpd.GeoDataFrame(df_filtered, geometry=gpd.points_from_xy(df_filtered.Longitude, df_filtered.Latitude))
        gdf.set_crs('EPSG:4326', inplace=True)  # Ensure the CRS is set to WGS 84

        print("Number of valid pixels: " + str(len(gdf)))

    # Rewrite the zip file including the modified target file
    with zipfile.ZipFile(file_zip_path, 'w', zipfile.ZIP_DEFLATED) as z:
        for foldername, subfolders, filenames in os.walk(process_directory):
            for filename in filenames:
                filepath = os.path.join(foldername, filename)
                arcname = os.path.relpath(filepath, process_directory)
                z.write(filepath, arcname)
                os.remove(filepath)  # Optionally remove the file after adding to zip

    # Optionally remove the processing directory after updating the zip
    shutil.rmtree(process_directory)
    print("Processing complete, directory cleaned.")

# Path and file specifications
year = "2018"
zip_path = f"C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\Landsat8\\{year}.zip"
satellite_image = "L8_UTC_20181218_031630"
target_file_path = f"{year}/{satellite_image}.tif"
processing_path = f"C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\Landsat8\\{year}_processing"

# Execute the function
unzip_process_zip(zip_path, target_file_path, processing_path)

All files extracted for processing.
Number of valid pixels: 2128012
Processing complete, directory cleaned.


#### Filtering specific locations to verify the effect of cool paints

A. Bukit Purmei Block 112 and 114

In [89]:
import geopandas as gpd

geojson_path = "C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\ADDRPT.geojson"
postal_code_112 = "090112"
postal_code_114 = "090114"

# Load the GeoJSON file
gdf = gpd.read_file(geojson_path)

# Function to retrieve coordinates by postal code
def get_coordinates_by_postal_code(postal_code):
    # Filter GeoDataFrame for the given postal code
    filtered_gdf = gdf[gdf['POSTAL_CODE'] == postal_code]
    if not filtered_gdf.empty:
        # Extract coordinates
        point = filtered_gdf.iloc[0].geometry
        return point.x, point.y
    else:
        return None, None

longitude_112, latitude_112 = get_coordinates_by_postal_code(postal_code_112)
longitude_114, latitude_114 = get_coordinates_by_postal_code(postal_code_114)

if longitude_112 and latitude_112 and longitude_114 and latitude_114:
    print(f'Coordinates for postal code {postal_code_112}: Longitude {longitude_112}, Latitude {latitude_112}')
    print(f'Coordinates for postal code {postal_code_114}: Longitude {longitude_114}, Latitude {latitude_114}')
else:
    print('Postal code not found.')

Coordinates for postal code 090112: Longitude 103.82593292805574, Latitude 1.2745285256209595
Coordinates for postal code 090114: Longitude 103.82588719010951, Latitude 1.2750718182249274


In [90]:
# Finding the central postal code for Blocks 112 and 114
if longitude_112 and latitude_112 and longitude_114 and latitude_114:
    # Calculate the average coordinates
    avg_longitude = (longitude_112 + longitude_114) / 2
    avg_latitude = (latitude_112 + latitude_114) / 2
    print(f'Average coordinates between postal codes {postal_code_112} and {postal_code_114}:')
    print(f'Longitude: {avg_longitude}, Latitude: {avg_latitude}')
else:
    print('Postal code not found.')

Average coordinates between postal codes 090112 and 090114:
Longitude: 103.82591005908262, Latitude: 1.2748001719229434


In [96]:
gdf = gpd.GeoDataFrame(df_filtered, geometry=gpd.points_from_xy(df_filtered.Longitude, df_filtered.Latitude))
gdf.set_crs('EPSG:4326', inplace=True)  # Ensure the CRS is set to WGS 84

# Define your point of interest and buffer distance in meters
poi = Point(avg_longitude, avg_latitude)  # Example: Central Singapore
desired_radius = 90
buffer = poi.buffer(desired_radius / 111320)  # Convert meters to degrees approximately

# Filter points within the buffer
filtered_gdf = gdf[gdf.geometry.within(buffer)]

# Save or process your filtered data
print(f"Number of points within {desired_radius}m radius: {len(filtered_gdf)}")

Number of points within 90m radius: 27


In [95]:
print(filtered_gdf.head())

          Longitude  Latitude   SR_B1   SR_B2   SR_B3   SR_B4    SR_B5  \
1280350  103.825495  1.275399  5970.0  6392.0  7446.0  7141.0   8943.0   
1280351  103.825765  1.275400  4182.0  4794.0  6465.0  5971.0   8331.0   
1280352  103.826035  1.275400  2221.0  3029.0  5449.0  4657.0   7380.0   
1280353  103.826304  1.275400  1396.0  2365.0  5162.0  4338.0   6961.0   
1282131  103.825226  1.275128  6831.0  7069.0  7909.0  7497.0  10591.0   

          SR_B6   SR_B7  SR_QA_AEROSOL  ...  SR_B7_Scaled  ST_ATRAN_Scaled  \
1280350  8517.0  8071.0            228  ...      0.021952           0.3657   
1280351  8218.0  7860.0            228  ...      0.016150           0.3657   
1280352  8038.0  7873.0            228  ...      0.016507           0.3657   
1280353  8166.0  7990.0            228  ...      0.019725           0.3657   
1282131  8931.0  8173.0            228  ...      0.024758           0.3657   

         ST_CDIST_Scaled  ST_DRAD_Scaled  ST_EMIS_Scaled  ST_EMSD_Scaled  \
1280350   

In [94]:
import geopandas as gpd
import hvplot.pandas
import holoviews as hv
from bokeh.palettes import Turbo256  # Import a predefined Bokeh palette
import panel as pn

# Assuming 'filtered_gdf' is your preloaded GeoDataFrame
# Ensure the CRS is set
if filtered_gdf.crs is None:
    filtered_gdf.set_crs('epsg:4326', inplace=True)  # Assuming the data is in WGS 84

# Convert to Web Mercator for better mapping support
filtered_gdf = filtered_gdf.to_crs(epsg=3857)

# Define a function to select a subset of the color palette
def select_colors(palette, n):
    return [palette[int(i)] for i in np.linspace(0, len(palette)-1, n)]

# Create a custom color scale using a continuous palette
custom_palette = select_colors(Turbo256, 256)  # More colors for smoother transitions

# Create the heatmap
heatmap = filtered_gdf.hvplot.points('Longitude', 'Latitude', geo=True, c='ST_B10_Celsius',
                                     cmap=custom_palette, size=5,
                                     tiles='OSM', frame_width=700, frame_height=500,
                                     colorbar=True, clim=(20, 40))

# Set up Panel to display the plot
pane = pn.panel(heatmap)
pane.save(f'C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\MSE-ES-UHI\\2_landsat\\Heatmaps\\{postal_code_112}_{satellite_image}_Land_Surface_Temperature_Map.html', embed=True)

# Display the plot in the notebook
pane.show()

Launching server at http://localhost:59410
